## Demo: Denoising of 3D mitochondrial outer membrane data

We extended the 2D Unet of SN2N to a 3D Unet to suit for three-dimensional XYZ data.  
In general, the core parameters of SN2N's 2D and 3D versions are essentially the same. While there are slight differences in specific parameters, they do not affect user usage. Users only need to grasp the core parameters to freely use SN2N for 2D and 3D data denoising tasks. For detailed parameters, please refer to SN2N.get_options.py.

<p>
<img src='./imgs/net.png' align="left" width=500>
</p>

## Raw data

The effectiveness of SN2N on 3D networks was validated using 3D OMM network imaging of live COS-7 cells labeled with Tom20-mCherry on SD-SIM sysytem.  
The raw data has a size of 101 * 1478 * 1137 pixels, with a pixel size of 38.23 nm, and is avaliable at https://zenodo.org/records/12697244/files/c12_SR_w1L-561_t1.tif?download=1.

<p>
<img src='./imgs/t3-raw-MAX_colored.png' align="left" width=200>
</p>

## Generating training data for SN2N

### Step 1: Data preprocessing

We first perform 3D volumetric <font color="red">percentaged normalization</font> on the data, along with optional background subtraction, to reduce pixel variations, eliminate hot pixels, and remove baseline signals.  
This is done to enhance the adaptability of the model to the input data.

In [2]:
import os
import sys
import tifffile
import numpy as np
sys.path.append(os.path.abspath(r'E:\PreStudy\02Source\SN2N'))
from SN2N.utils import *

flage = 1
datapath_list = []
pmin = 30
pmax = 99.95
imgpath = './data/raw_data/'
save_path = ('./data/raw_data_per/')

os.makedirs(save_path, exist_ok=True)
for (root, dirs, files) in os.walk(imgpath):
    for j, Ufile in enumerate(files):
        img_path = os.path.join(root, Ufile)
        datapath_list.append(img_path)
          
l = len(datapath_list)
for ll in range(l):
    image_data = tifffile.imread(datapath_list[ll])
    [t, x, y] = image_data.shape
    image_data_new = normalize_percentage(x = image_data, pmin = pmin, pmax = pmax, axis=None, clip=True, eps=1e-20, dtype=np.float32)
    image_data_new2 = 255*image_data_new
    tifffile.imsave(('%s/%d.tif') %(save_path, flage), image_data_new2.astype('uint8'))
    flage = flage + 1
    
print("Data preprocessing is completed.")

Data preprocessing is completed.


### Step 2: Define custom parameters.

We first need to use raw_data to generate SN2N data pairs.  
When generating data pairs, we need to first confirm and <font color="red">set important parameters</font> related to data generation.

-----Parameters------  
<font color="red">=====Important==========</font>  
- <span style="color:blue">img_path:</span>  
    Path of raw images to train.  
- <span style="color:blue">P2Pmode(0 ~ 3):</span>  
    Augmentation mode for Patch2Patch.  
    0: NONE;  
    1: Direct interchange in t;  
    2: Interchange in single frame;  
    3: Interchange in multiple frame but in different regions;  
    {default: 0}  
- <span style="color:blue">P2Pup:</span>  
    Increase the dataset to its (1 + P2Pup) times size.  
    {default: 0}  
- <span style="color:blue">BAmode(0 ~ 2)</span>  
    Basic augmentation mode.  
    0: NONE;  
    1: double the dataset with random rotate&flip;  
    2: eightfold the dataset with random rotate&flip;  
    {default: 0}  
- <span style="color:blue">SWsize</span>  
    Interval pixel of sliding window for generating image pathes.  
    {default: 64}  
    
<font color="red">======Other parameters do not require modification; for details, refer to SN2N.get_options.datagen3D.========</font> 

In [3]:
# import sys
from SN2N.datagen import generator3D
from SN2N.get_options import datagen3D

img_path = './data/raw_data_per'
P2Pmode = '0'
P2Pup = '0'
BAmode = '0'
SWsize = '64' 
SWfilter = '15'


datagen3D_args = [
    '--img_path', img_path,
    '--P2Pmode', P2Pmode,
    '--P2Pup', P2Pup,
    '--BAmode', BAmode,
    '--SWsize', SWsize,
    '--SWfilter', SWfilter
]

args = datagen3D(datagen3D_args)
print("Arguments received:", args)

Arguments received: Namespace(BAmode=0, P2Pmode=0, P2Ppatch='64', P2Pup=0, SWfilter=15, SWmode=1, SWsize=64, ifx2=True, img_path='./data/raw_data_per', inter_mode='Fourier', vol_patch='16,128,128')


### Step 3: Execute data generation.

The <font color="red">raw data</font> used for generating training data is stored in the <font color="red">'Path:/raw_data'</font> directory, and the generated <font color="red">training data pairs</font> are by default stored in the <font color="red">'Path:/datasets'</font>

In [4]:
d = generator3D(img_path=args.img_path, P2Pmode=args.P2Pmode, P2Pup=args.P2Pup, BAmode=args.BAmode, SWsize=args.SWsize, SWfilter=args.SWfilter)

d.execute()
print("Data generation completed.")

The path for the raw images used for training is located under:
./data/raw_data_per
The training dataset is being saved under:
./data\datasets
For number 1 frame
Saving training images: 100
Saving training images: 200
Saving training images: 300
Saving training images: 400
Saving training images: 500
Saving training images: 600
Saving training images: 700
Saving training images: 800
Saving training images: 900
Saving training images: 1000
Saving training images: 1100
Saving training images: 1200
Saving training images: 1300
Saving training images: 1400
Saving training images: 1500
Saving training images: 1600
Saving training images: 1700
Saving training images: 1800
Saving training images: 1900
Saving training images: 2000
Saving training images: 2100
Saving training images: 2200
Saving training images: 2300
Saving training images: 2400
Saving training images: 2500
Saving training images: 2600
Saving training images: 2700
Saving training images: 2800
Saving training images: 2900
Saving

## Training data

We need to use training data pairs to train SN2N model.  
When training, we need to first confirm and <font color="red">set important parameters</font> related to trainers.

-----Parameters------  
<font color="red">=====Important==========</font>  
- <span style="color:blue">img_path:</span>  
    Path of raw images to train.  
- <span style="color:blue">sn2n_loss:</span>  
    Weight of self-constrained loss.  
    <span style="color:red">One of the most crucial parameters that needs adjustment， generally set to 0~2.</span>  
    A higher sn2n_loss weight implies relatively better denoising and more pronounced smoothing effects.  
    {default: 1}  
- <span style="color:blue">bs:</span>  
    Training batch size.  
    {default: 4}  
- <span style="color:blue">lr</span>  
    Learning rate.       
    {default: 2e-4}  
- <span style="color:blue">epochs</span>  
    Total number of training epochs.  
    {default: 20}  
    
<font color="red">======Other parameters do not require modification; for details, refer to SN2N.get_options.trainer3D.========</font>  

### Step 1: Define custom parameters.

In [5]:
from SN2N.trainer import net3D
from SN2N.get_options import trainer3D

img_path = './data/raw_data_per'
sn2n_loss = '0.5'
bs = '4'
lr = '2e-4'
# epochs = '20'
epochs = '1'

trainer3D_args = [
    '--img_path', img_path,
    '--sn2n_loss', sn2n_loss,
    '--bs', bs,
    '--lr', lr,
    '--epochs', epochs
]

args = trainer3D(trainer3D_args)
print("Arguments received:", args)

Arguments received: Namespace(bs=4, epochs=1, if_alr=True, img_path='./data/raw_data_per', lr=0.0002, sn2n_loss=0.5, vol_patch='16,128,128')


### Step 2: Execute training.

The <font color="red">raw data</font> used for generating training data is stored in the <font color="red">'Path:/raw_data'</font> directory  
the <font color="red">models</font> are by default stored in the <font color="red">'Path:/models'</font>  
During the training process, we also performed real-time predictions of images to monitor the model training progress. 
For 3D tasks, to avoid memory overflow during image prediction, we utilized the (0:16, 0:256, 0:256) portion of the raw image for prediction.  
The <font color="red">images</font> results are by default stored in the <font color="red">'Path:/images'</font> directory.

In [6]:
sn2nunet = net3D(img_path = args.img_path, sn2n_loss = args.sn2n_loss, bs = args.bs, lr = args.lr, epochs = args.epochs)
sn2nunet.train()
print("Training completed.")

d:\Software\anaconda3\envs\SN2N\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


The path for the raw images used for training is located under:
./data/raw_data_per
The training dataset is being saved under:
./data\datasets
Models is being saved under:
./data\models
Training temporary prediction images is being saved under:
./data\images
[Epoch 0/1] [Batch 0/1075] [loss:32.344675] time:0:00:04.616272
[Epoch 0/1] [Batch 1/1075] [loss:29.729965] time:0:00:05.601182
[Epoch 0/1] [Batch 2/1075] [loss:27.590355] time:0:00:06.267509
[Epoch 0/1] [Batch 3/1075] [loss:25.875315] time:0:00:06.933468
[Epoch 0/1] [Batch 4/1075] [loss:23.012570] time:0:00:07.599660
[Epoch 0/1] [Batch 5/1075] [loss:20.084064] time:0:00:08.249989
[Epoch 0/1] [Batch 6/1075] [loss:20.033109] time:0:00:08.918450
[Epoch 0/1] [Batch 7/1075] [loss:21.963970] time:0:00:09.584348
[Epoch 0/1] [Batch 8/1075] [loss:19.517794] time:0:00:10.251184
[Epoch 0/1] [Batch 9/1075] [loss:18.779343] time:0:00:10.933483
[Epoch 0/1] [Batch 10/1075] [loss:17.951301] time:0:00:11.623153
[Epoch 0/1] [Batch 11/1075] [loss:17